In [ ]:
#Upload File
from google.colab import files
data_to_load = files.upload()

In [ ]:
#library
from math import log
from collections import defaultdict
import json
import pprint
import csv
import pandas as pd

In [ ]:
filename = "testbmi.csv"

In [33]:
df = pd.read_csv('testbmi.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Gender  500 non-null    int64 
 1   Height  500 non-null    int64 
 2   Weight  500 non-null    int64 
 3    Index  500 non-null    object
dtypes: int64(3), object(1)
memory usage: 15.8+ KB


In [ ]:
'''
For gender:
0 = Female
1= Male
'''
df.head()

,Gender,Height,Weight,Index
0,1,174,96,indeks 4
1,1,189,87,indeks 2
2,0,185,110,indeks 4
3,0,195,104,indeks 3
4,1,149,61,indeks 3


In [ ]:
def create_Dataset():
  features = []
  dataset = []
  with open(filename, 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    features = next(csvreader)
    
    for row in csvreader:
      dataset.append(row)
  return dataset, features

In [ ]:
def calculate_Entropy(dataset):
  counter = defaultdict(int)
  for record in dataset:
    label = record[-1]
    counter[label] += 1
  entropy = 0.0
  for key in counter:
    probability = counter[key]/len(dataset)
    entropy -= probability * log(probability, 2)
  return entropy

In [ ]:
def split_Dataset(dataSet, axis, value):
  retDataSet = []
  for featVec in dataSet:
    if featVec[axis] == value:
      reducedFeatVec = featVec[:axis]
      reducedFeatVec.extend(featVec[axis+1:])
      retDataSet.append(reducedFeatVec)
  return retDataSet

In [ ]:
def chooseBestFeat_toSplit(dataset):
  baseEntropy = calculate_Entropy(dataset)
  bestInfoGain = 0.0; bestFeature = -1

  numFeat = len(dataset[0]) - 1
  for indx in range(numFeat):
    featValues = {record[indx] for record in dataset}
    featEntropy = 0.0
    for value in featValues:
      subDataset = split_Dataset(dataset, indx, value)
      probability = len(subDataset)/float(len(dataset))
      featEntropy += probability * calculate_Entropy(subDataset)
    
    infoGain = baseEntropy - featEntropy
    if infoGain > bestInfoGain:
      bestInfoGain = infoGain
      bestFeature = indx
  return bestFeature

In [ ]:
def create_Tree(dataset, features):
  labels = [record[-1] for record in dataset]

  if labels.count(labels[0]) == len(labels):
    return labels[0]
  if len(dataset[0]) == 1:
    mjcount = max(labels, key=labels.count)
    return (mjcount)
  
  bestFeat = chooseBestFeat_toSplit(dataset)
  bestFeatLabel = features[bestFeat]
  featValues = {record[bestFeat] for record in dataset}
  subLabels = features[:]
  del(subLabels[bestFeat])

  myTree = {bestFeatLabel:{}}
  for value in featValues:
    subDataset = split_Dataset(dataset, bestFeat, value)
    subTree = create_Tree(subDataset, subLabels)
    myTree[bestFeatLabel].update({value: subTree})
  return myTree

In [48]:
def predict(inputTree, features, testVec):
  def classify(inputTree, testDict):
    (key, subtree), = inputTree.items()
    testValue = testDict.pop(key)
    print(key)
    print(testValue)

    if len(subtree[testValue]) != 1:
      #print(subtree[testValue])
      return subtree[testValue]
    else:
      #print(subtree[testValue])
      return classify(subtree[testValue], testDict)
  
  testDict = dict(zip(features, testVec))
  return classify(inputTree, testDict)

In [53]:
def main(testData):
  dataset, features = create_Dataset()
  tree = create_Tree(dataset, features)
  #pprintTree(tree)
  
  for vec in testData:
    pred = predict(tree, features, vec)
    if(pred == 'indeks 0'):
        print("\nFrom sample data given, its predicted that you are EXTREMELY WEAK")
    elif(pred == 'indeks 1'):
        print("\nFrom sample data given, its predicted that you are WEAK")
    elif(pred == 'indeks 2'):
        print("\nFrom sample data given, its predicted that you are NORMAL")
    elif(pred == 'indeks 3'):
        print("\nFrom sample data given, its predicted that you are OVERWEIGHT")
    elif(pred == 'indeks 4'):
        print("\nFrom sample data given, its predicted that you are OBESITY")
    elif(pred == 'indeks 5'):
        print("\nFrom sample data given, its predicted that you are EXTREMELY OBESITY")

In [54]:
testData = [('1', '174', '96',)]
#testData = [('0', '195', '65',)]
#testData = [('1', '181', '78',)]
#testData = [('1', '176', '87',)]
#testData = [('0', '160', '109',)]
main(testData)

Weight
96
Height
174

 From sample data given, its predicted that you are OBESITY


In [ ]:
'''
datas, feat = create_Dataset()
sampleData = [1]*3

count = 0
for i in feat:
  print("What is your ",i,"? 1/0 \n 1) Male\n 2) Female")
  sampleData[count]=input()
  count+=1
main(sampleData)
'''